In [95]:
%matplotlib inline
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter
from cleantext import clean
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from collections import Counter
import subprocess
import zipfile
import io
import csv
import zipfile38 as zipfile
from dask import dataframe as dd
from sklearn.model_selection import train_test_split
import h5py
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/simonmork/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
def clean_text(text):
    """Calls the clean function from cleantext clean on a string"""
    t = clean(text,
    fix_unicode=False,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=True,         # fully strip line breaks as opposed to only normalizing them NOT WORKING?
    no_urls=True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=False,         # replace all phone numbers with a special token
    no_numbers=True,               # replace all numbers with a special token
    no_digits=False,                # replace all digits with a special token
    no_currency_symbols=False,      # replace all currency symbols with a special token
    no_punct=True,                 # remove punctuations
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="URL",
    replace_with_email="EMAIL",
    replace_with_phone_number="PHONE",
    replace_with_number="NUM",
    replace_with_digit="0",
    replace_with_currency_symbol="CUR",
    lang="en"                       # set to 'de' for German special handling
    )
    return t


print("Getting csv...")
pd.set_option("display.max_colwidth", 10000000)

df = pd.read_csv('https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv')

#file_path = 'news.csv.zip'
# Open the zip file and read the contents of the csv file
#with zipfile.ZipFile(file_path) as zip_file:
    #with zip_file.open('news.csv') as csv_file:
        # Read the csv file using pandas read_csv function
       # df = pd.read_csv(csv_file)
    
print("Done")    
    


Getting csv...
Done


In [65]:

print("Getting headers..")
column_names = list(df.columns.values)
#print(column_names)

print("Cleaning...")
for index, row in df.iterrows():
    #print(df.at[index,'content'])
    cleaned = clean_text(row['content'])
    df.at[index,'content'] = cleaned
    #print(cleaned)
    
    
# Split the content column into a list of words
words = df['content'].str.split()
# Create a set of unique words
unique_words = set(word for word_list in words for word in word_list)
# Count the number of unique words
num_unique_words_after_clean = len(unique_words)
print(num_unique_words_after_clean) 



print("Tokenizing and removing stopwords...")
stop_words = set(stopwords.words('english'))
for index, row in df.iterrows():    
    #print(df.at[index,'content'])
    df.at[index,'content'] = nltk.word_tokenize(row['content'])
    
    filtered_sentence = []
    for w in df.at[index,'content']:
        if w not in stop_words:
            filtered_sentence.append(w)
    
    df.at[index,'content'] = filtered_sentence
    #print(filtered_sentence)
 

# concatenate all the lists into a single list of words
words = [word for content in df['content'] for word in content]
# get the set of unique words
unique_words = set(words)
# get the number of unique words
num_unique_words_after_stop = len(unique_words)
print(num_unique_words_after_stop)
print(1-num_unique_words_after_stop/num_unique_words_after_clean)
    
    

print("Stemming")
snowball = SnowballStemmer(language='english')
for index, row in df.iterrows():   
    stemmed = []
    for word in df.at[index,'content']:
        stemmed.append(snowball.stem(word))
    df.at[index,'content'] = stemmed
    #print(stemmed)


# concatenate all the lists into a single list of words
words = [word for content in df['content'] for word in content]
# get the set of unique words
unique_words = set(words)
# get the number of unique words
num_unique_words_after_stem = len(unique_words)
print(num_unique_words_after_stem)
print(1-num_unique_words_after_stem/num_unique_words_after_stop)


print("Done...")

Getting headers..
['Unnamed: 0', 'id', 'domain', 'type', 'url', 'content', 'scraped_at', 'inserted_at', 'updated_at', 'title', 'authors', 'keywords', 'meta_keywords', 'meta_description', 'tags', 'summary']
Cleaning...
16683
Tokenizing and removing stopwords...
16546
0.008211952286758972
Stemming
11022
0.3338571255892663
Done...


In [66]:
#Task 2
print("Counting number of urls, dates and num, and fake news and trump")
url = 0
num = 0
trymp=0
fakenews = 0
notfake = 0
for index, row in df.iterrows():
    #print(data['content'])
    for word in df.at[index,'content']:
        #print(word)
        if word == 'url':
            url = url+1
        if word == 'num':
            num = num+1
        if word == 'trump':
            trymp = trymp+1
    if df.at[index,'type'] == 'fake':
        fakenews = fakenews+1
    else:
        notfake = notfake+1
print("Num urls: ", url)
print("Num num: ", num)
print("Num trymp: ", trymp)
print("Num fake: ", fakenews)
print("Num notfake: ", notfake)

Counting number of urls, dates and num, and fake news and trump
Num urls:  240
Num num:  1943
Num trymp:  348
Num fake:  155
Num notfake:  95


In [67]:
#Task 4

X, y = df.content, df['type']

#print(X)
#print("y: ", y)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 0)

print("train: ")
print(X_train, y_train)


print("test: ")
print(X_test, y_test)

print("Validation: ")
print(X_val, y_val)



train: 
71                                                                                                         [subscrib, canada, free, press, free, chanc, learn, practic, limit, manner, control, condit, optim, steer, wheel, control, vision, anticip, well, obstacl, avoid, emerg, brake, tactic, straight, line, curv, mitsubishi, winter, drive, event, help, teach, control, snow, winter, may, way, outsoon, there, still, enough, old, person, around, ensur, snowi, canadian, road, challeng, least, week, mean, there, still, ampl, opportun, skidrel, fender, bender, spring, spring, grass, start, rizz, end, mitsubishi, canada, road, well, bring, profession, driver, outland, suvcrossov, alberta, showcas, crummi, weather, capabl, vehicl, give, handi, tip, attende, find, oot, aboot, white, stuff, stick, love, event, theyr, fun, help, navig, neig, confid, skill, ...]
160    [worldwid, laser, technolog, market, analysi, report, num, challeng, opportun, headlin, bitcoin, blockchain, search, exceed,

In [91]:
#Convert y_train to binary and save in y_train_rel. Either reliable or fake
y_train_rel = y_train
rel = 0
fake = 0
for entry in y_train_rel:
    if entry != 'fake':
        entry = 'reliable'
        rel = rel +1
    else:
        fake = fake+1
print(rel)
print(fake)
y_train_rel = np.asarray(y_train_rel)

82
118


In [99]:
X_train_str = [' '.join(words) for words in X_train]
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train_str)
print(X_train_counts)

  (0, 8454)	1
  (0, 1256)	4
  (0, 3403)	2
  (0, 6743)	1
  (0, 1407)	3
  (0, 4864)	3
  (0, 6686)	7
  (0, 4982)	2
  (0, 5198)	2
  (0, 1855)	14
  (0, 1763)	3
  (0, 6137)	2
  (0, 8321)	3
  (0, 9665)	10
  (0, 9478)	2
  (0, 435)	2
  (0, 9632)	10
  (0, 6020)	2
  (0, 667)	3
  (0, 2745)	2
  (0, 1074)	8
  (0, 8616)	2
  (0, 8382)	3
  (0, 4987)	7
  (0, 2057)	2
  :	:
  (198, 5279)	2
  (198, 2404)	1
  (198, 1792)	3
  (198, 1794)	1
  (198, 1793)	1
  (198, 875)	1
  (198, 2540)	1
  (198, 9119)	1
  (198, 2030)	1
  (198, 2945)	1
  (198, 9120)	1
  (198, 6279)	1
  (198, 3873)	1
  (198, 6554)	1
  (198, 7422)	1
  (198, 8407)	1
  (198, 9350)	1
  (199, 2775)	1
  (199, 2112)	2
  (199, 8836)	1
  (199, 2353)	1
  (199, 9161)	1
  (199, 5059)	1
  (199, 2642)	1
  (199, 1489)	1


In [93]:
y_train_rel = np.array(y_train_rel)
X_train = np.array(X_train)
for elm in X_train:
    np.array(elm)
for elm in y_train:
    np.array(elm)
for elm in X_val:
    np.array(elm)

#finding the best C value
C_values = [0.001, 0.1, 1, 100]
for C in C_values:
    #training the model
    simple_log_model = LogisticRegression(max_iter=100000,C=C)
    print(y_train_rel)
    simple_log_model.fit(X_train, y_train_rel)
    y_pred = simple_log_model.predict(np.asarray(X_val))
    print(y_pred)

['conspiracy' 'fake' 'fake' 'fake' 'political' 'fake' 'fake' 'conspiracy'
 'fake' 'reliable' 'fake' 'political' 'fake' 'fake' 'fake' nan
 'conspiracy' 'fake' 'fake' 'fake' nan 'fake' 'fake' 'unreliable' 'fake'
 'fake' 'fake' 'clickbait' 'fake' 'fake' 'fake' 'political' 'conspiracy'
 'fake' 'fake' nan 'unknown' 'fake' 'fake' 'unreliable' 'political'
 'conspiracy' 'fake' 'unknown' 'political' 'fake' 'fake' 'fake'
 'conspiracy' 'fake' 'fake' 'political' 'conspiracy' 'fake' 'conspiracy'
 'unreliable' nan 'fake' 'junksci' 'political' 'fake' 'political'
 'unknown' 'fake' 'junksci' 'hate' 'fake' 'fake' 'fake' 'conspiracy'
 'fake' 'conspiracy' 'fake' 'fake' 'fake' 'fake' 'fake' 'fake' 'fake'
 'bias' 'fake' 'fake' 'conspiracy' 'bias' 'fake' 'fake' 'fake' 'junksci'
 'fake' 'fake' 'fake' 'fake' 'fake' 'fake' 'unreliable' 'political'
 'political' 'conspiracy' 'political' 'conspiracy' 'fake' nan 'fake'
 'fake' 'political' 'fake' 'fake' 'fake' 'fake' 'unreliable' nan
 'conspiracy' 'fake' 'fake' nan 

ValueError: setting an array element with a sequence.

In [ ]:
#tips: nearest neighbor on missing data